<a href="https://colab.research.google.com/github/YuMan-Tam/deep-hedging-demo/blob/master/deep_hedging_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title <font color='Blue'>**Overheads**</font>

# Author: Yu-Man Tam
# Last updated: 3/3/2020

# Reference: Deep Hedging (2019, Quantitative Finance) by Buehler et al.
# https://www.tandfonline.com/doi/abs/10.1080/14697688.2019.1571683

import sys, os
sys.path.insert(0, os.getcwd() + "/lib")

# Linear algebra, finance, and machine learning libraries
import numpy as np
import QuantLib as ql
import tensorflow as tf

# Avoid X11-server IO error.
# https://matplotlib.org/faq/howto_faq.html
# matplotlib.use('Agg') 

from itertools import repeat
import pickle
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
import tensorflow.keras.backend as K

# For plots and presentation
from tqdm import trange, tqdm
from tqdm.keras import TqdmCallback
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# For Jupyter notebooks
from IPython.display import clear_output
from IPython.core.display import display, HTML

from Stochastic_Processes import BlackScholesProcess
from Utilities import train_test_split, reshape_1D
from Deep_Hedging import Deep_Hedging
from Validation import Validation

# Tensorflow settings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.compat.v1.disable_eager_execution()

%load_ext autoreload
%matplotlib inline

clear_output()

print("\nFinish loading all necessary libraries!\n")

**Import all neccessary python, quantitative finance, and machine learning software libraries.**

In [0]:
#@title <font color='Blue'>**User Inputs**</font>

# Geometric Brownian Motion.
N = 30 # Number of time steps (in days)

S0 = 100.0 # Stock price at time = 0
sigma = 0.2 # Implied volatility
risk_free = 0.0 # Risk-free rate
dividend = 0.0 # Continuous dividend yield

Ktrain = 1*(10**5) # Size of training sample.
Ktest_ratio = 0.2 # Fraction of training sample as testing sample.

# European call option (short).
strike = S0
payoff_func = lambda x: -np.maximum(x - strike, 0.0)
calculation_date = ql.Date.todaysDate()
maturity_date = ql.Date.todaysDate() + N

# Day convention.
day_count = ql.Actual365Fixed() # Actual/Actual (ISDA)

# Proportional transaction cost.
epsilon = np.power(2.0,-8)*0.0                 

# Information set (in string)
# Choose from: S, log_S, normalized_log_S (by S0)
information_set = "normalized_log_S"

# Loss function
# loss_type = "CVaR" (Expected Shortfall) -> loss_param = alpha 
# loss_type = "Entropy" -> loss_param = lambda
loss_type = "Entropy"
loss_param = 1.0

# Neural network (NN) structure
m = 15 # Number of neurons in each hidden layer.
d = 3 # Number of hidden layers (Note including input nor output layer)         

# Other NN parameters
lr = 1e-2 # Learning rate
batch_size=256 # Batch size
epochs=50 # Number of epochs

use_batch_norm = False
kernel_initializer = "he_uniform"

activation_dense = "leaky_relu"
activation_output = "sigmoid"
final_period_cost = False

# Other control flags for development purpose.
mc_simulator = "QuantLib" # "QuantLib" or "Numpy"

flag_load_simulation_data = True
flag_load_model_simple = True
flag_load_model_recurrent = False
flag_load_BS_delta = True
flag_load_BS_asym = True
flag_show_fig = False
flag_plot_S_range = "uniform" # "uniform" or "percentile"
flag_load_PnL_DH_Simple_list = True
flag_presenation_mode = True

# Output directories
simulation_data_dir = "./data/"
subdir_hyper_param = "/".join((str(epsilon),activation_dense,activation_output,str(m),str(d),str(batch_size),""))
model_data_dir ="./data/" + subdir_hyper_param
fig_dir = "./figure/"+ subdir_hyper_param

print("Finish loading all user inputs!\n")

**Provide input parameters for Monte Carlo simulation, call option, transaction cost, loss function, and deep hedging algorithm.**

In [0]:
#@title <font color='Blue'>**Monte Carlo Simulation - Generate Random Paths of Stock Prices**</font>

# Create a directory if the output directories do not exist.
for dir in (simulation_data_dir, model_data_dir, fig_dir):
	try:
		os.makedirs(dir)
	except:
		pass
		
# Length of one time-step (as fraction of a year).
nobs = int(Ktrain*(1+Ktest_ratio)) # Training + Testing

dt = day_count.yearFraction(calculation_date,calculation_date + 1) 
maturity = N*dt # Maturities (in the unit of a year)

stochastic_process = BlackScholesProcess(s0 = S0, sigma = sigma, \
                      risk_free = risk_free, dividend = dividend, day_count=day_count)

simulation_data_file = simulation_data_dir +"stock_" + str(int(np.log10(Ktrain))) + ".npy"
if not flag_load_simulation_data:
	if mc_simulator is "QuantLib":
		S = stochastic_process.gen_path(maturity, N, nobs)
	elif mc_simulator is "Numpy":
		# Only for geometric Brownian motion
		randn = (risk_free - dividend - sigma ** 2 / 2.0) * dt + \
								sigma * np.random.normal(0, np.sqrt(dt), size=(nobs, N))
		S = np.hstack((np.ones((nobs,1))*np.log(S0),randn))
		S = np.exp(np.cumsum(S,axis=1))
		
	np.save(simulation_data_file, S)
else:	
	try:
	  print("Check if there is usable data from previous simulation...\n")
	  S = np.load(simulation_data_file)
	  print("Successfully loaded data!!!\n")
	except:
		print("There is no simulation data!!!")

In [0]:
#@title <font color='Blue'>**Prepare data to be fed into the deep hedging algorithm.**</font>
payoff_T = payoff_func(S[:,-1]) # Payoff of the call option

trade_set =  np.stack((S),axis=1) # Trading set

if information_set is "S":
  I =  np.stack((S),axis=1) # Information set
elif information_set is "log_S":
  I =  np.stack((np.log(S)),axis=1)
elif information_set is "normalized_log_S":
	I =  np.stack((np.log(S/S0)),axis=1)
	
# Structure of xtrain:
#   1) Trade set: [S]
#   2) Information set: [S] 
#   3) payoff (dim = 1)
x_all = []
for i in range(N+1):
	x_all += [trade_set[i,:]]
	if i != N:
		x_all += [I[i,:]]
x_all += [payoff_T]

# Split the entire sample into a training sample and a testing sample.
test_size = int(Ktrain*Ktest_ratio)
[xtrain, xtest] = train_test_split(x_all, test_size=test_size)
[S_train, S_test] = train_test_split([S], test_size=test_size)
[option_payoff_train, option_payoff_test] = \
    train_test_split([x_all[-1]], test_size=test_size)

print("Finish preparing training and testing data!")

In [0]:
#@title <font color='Blue'>**Run the Deep Hedging Algorithm (Simple Network)!**</font>
%autoreload 2
strategy_type = "simple"
optimizer = Adam(learning_rate=lr)

# Setup and compile the model
model_simple = Deep_Hedging(N=N, d=d+2, m=m, risk_free=risk_free, \
                      maturity=maturity).model( \
                      strategy_type=strategy_type, epsilon = epsilon, \
                      loss_type=loss_type, loss_param = loss_param, \
											use_batch_norm = use_batch_norm, kernel_initializer = kernel_initializer, \
											activation_dense = activation_dense, activation_output = activation_output, \
											final_period_cost = final_period_cost)
model_simple.compile(optimizer=optimizer)

# # Stopping criteria
train_model_file = model_data_dir + \
			"_".join(("model", strategy_type,str(epsilon), \
			          "{epoch}", "{batch}"))
best_model_file = model_data_dir + "_".join(("model", strategy_type,str(epsilon)))

early_stopping = EarlyStopping(monitor="loss", \
          patience=5, min_delta=1e-4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="loss", \
          factor=0.5, patience=2, min_delta=1e-3, verbose=0)
model_checkpoint = ModelCheckpoint(train_model_file, \
          monitor="loss", save_best_only=False, \
          save_weights_only=True, save_freq = 1, verbose=0)
callbacks = [early_stopping, reduce_lr, model_checkpoint, TqdmCallback()]

# Try to preload model weights if possible.
if flag_load_model_simple:
	try:
		print("Loading model weights with simple network...\n")
		# As of Tensorflow 2.1, there is an error saving the entire model.
		# This option does not allow us to save the optimizer states. (ToDo)
		model_simple.load_weights(best_model_file)
		print("Finished loading model weights with simple network!\n")
	except:
		print("No saved model with simple network and no transaction cost.\n")
else:
	# Fit the model.
	model_fit = model_simple.fit(x=xtrain, batch_size=batch_size, epochs=epochs, \
				validation_data=(xtest, np.empty(0)), callbacks=callbacks, \
				verbose=0)
	model_simple.save_weights(best_model_file)
	print("Finished running deep hedging algorithm! (Simple Network)")

In [0]:
#@title <font color='Blue'>**Run the Deep Hedging Algorithm (Recurrent Network)!**</font>
strategy_type = "recurrent"
optimizer = Adam(learning_rate=lr)

# Setup and compile the model
model_recurrent = Deep_Hedging(N=N, d=d+2, m=m, risk_free=risk_free, \
										maturity=maturity).model( \
										strategy_type=strategy_type, epsilon = epsilon, \
										loss_type=loss_type, loss_param = loss_param, \
										use_batch_norm = use_batch_norm, kernel_initializer = kernel_initializer, \
										activation_dense = activation_dense, activation_output = activation_output, \
										final_period_cost = final_period_cost)
model_recurrent.compile(optimizer=optimizer)

# Stopping criteria
train_model_file = model_data_dir + \
			"_".join(("model", strategy_type,str(epsilon), \
			          "{epoch}", "{batch}"))
best_model_file = model_data_dir + "_".join(("model", strategy_type,str(epsilon)))

early_stopping = EarlyStopping(monitor="loss", \
				patience=5, min_delta=1e-4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="loss", \
				factor=0.5, patience=2, min_delta=1e-3, verbose=0)
model_checkpoint = ModelCheckpoint(train_model_file, \
          monitor="loss", save_best_only=False, \
          save_weights_only=True, save_freq = 1, verbose=0)

callbacks = [early_stopping, reduce_lr, model_checkpoint, TqdmCallback()]

# Try to preload model weights if possible.
if flag_load_model_recurrent:
	try:
		print("Loading model weights with simple network...\n")
		# As of Tensorflow 2.1, there is an error saving the entire model.
		# This option does not allow us to save the optimizer states. (ToDo)
		model_recurrent.load_weights(best_model_file)
		print("Finished loading model weights with simple network!\n")
	except:
		print("No saved model with recurrent network and no transaction cost.\n")
else:
	# Fit the model.
	model_fit = model_recurrent.fit(x=xtrain, batch_size=batch_size, epochs=epochs, \
				validation_data=(xtest, np.empty(0)), callbacks=callbacks, \
				verbose=0)
	model_recurrent.save_weights(best_model_file)
print("Finished running deep hedging algorithm! (Recurrent Network)")

In [0]:
#@title <font color='Blue'>**Results: Option Prices**</font>
report = Validation(model = model_simple, data = xtest, N = N, process = stochastic_process)

# Specify the instrument as an European call option
instrument = report.get_instrument(name = "European_Call", strike = strike, \
                                    maturity_date = maturity_date)

BS_price = report.get_model_PV(instrument, s0=S0)
risk_neutral_price = report.get_risk_neutral_PV(risk_free,dt,N)
nn_simple_price = model_simple.evaluate(xtest, batch_size=test_size, verbose=0)
nn_recurrent_price = model_recurrent.evaluate(xtest, batch_size=test_size, verbose=0)

print("The Black-Scholes model price is %2.3f." % BS_price)
print("The Risk Neutral price is %2.3f." % risk_neutral_price)
print("The Deep Hedging (with simple network) price is %2.3f." % nn_simple_price)
print("The Deep Hedging (with recurrent network) price is %2.3f." % nn_recurrent_price)

In [0]:
#@title <font color='Blue'>**Results: Black-Scholes PnL vs Deep Hedging PnL**</font>

# Compute Black-Scholes delta for each sample path at each time t.
BS_delta_file = simulation_data_dir + "BS_delta.npy"
if flag_load_BS_delta:
	try:
		model_delta = np.load(BS_delta_file)
	except:
		print("No saved Black-Scholes model delta!!!\n")
		print("Proceed to create Black-Scholes model delta!!!\n")
		flag_load_BS_delta = False
		pass

if not flag_load_BS_delta:
	model_delta = np.zeros((test_size, N+1))
	func_delta = lambda i, j: report.get_model_delta(instrument, \
									s0=S_test[0][i,j], calculation_date=calculation_date + j)

	for j in trange(N+1):
			model_delta[:,j]  = list(map(func_delta, range(test_size), \
															repeat(j,test_size)))
	np.save(BS_delta_file, model_delta)
	
# Compute Black-Scholes PnL (for a short position, i.e. the Bank sells
# a call option. The model delta from Quantlib is a long delta.
PnL_BS = (np.multiply(S_test[0][:,0], -model_delta[:,0]) - \
          np.abs(model_delta[:,0])*S_test[0][:,0]*epsilon)* \
          np.exp(risk_free*dt)
for t in range(1, N):
  PnL_BS += np.multiply(S_test[0][:,t], -model_delta[:,t] + model_delta[:,t-1]) - \
               np.abs(model_delta[:,t] -model_delta[:,t-1])*S_test[0][:,t]*epsilon
  PnL_BS = PnL_BS*np.exp(risk_free*dt)

PnL_BS += (np.multiply(S_test[0][:,N],model_delta[:,N-1]) + option_payoff_test[0] - \
              np.abs(model_delta[:,N-1])*S_test[0][:,N]*epsilon)

# Compute deep hedging PnL.
PnL_DH = model_simple.predict(xtest,batch_size=test_size, verbose=0)

# Plot Black-Scholes PnL and Deep Hedging PnL (with BS_price charged on both).
fig_PnL = plt.figure(dpi= 125, facecolor='w')
fig_PnL.suptitle("Black-Scholes PnL vs Deep Hedging PnL \n", \
      fontweight="bold")
ax = fig_PnL.add_subplot()
ax.set_title("Simple Network Structure with epsilon = " + str(epsilon), \
      fontsize=8)
ax.set_xlabel("PnL")
ax.set_ylabel("Frequency")

plt.close("all")
if flag_presenation_mode:
	strategy_type = "simple"
	num_batch_per_epoch = int(Ktrain/batch_size)
	# In presentation mode, we present the DH PnL for each epoch.
	# First, load the PnL for each epoch from the model checkpoints.
	PnL_list_file = model_data_dir + \
			"_".join(("model", strategy_type,str(epsilon),"PnL_data.npy"))
	
	if flag_load_PnL_DH_Simple_list:
		try:
			print("Loading Deep Hedge PnL for each batch and epoch...")
			PnL_DH_Simple_data = np.load(PnL_list_file)
		except:
			print("PnL_DH_Simple_list does not exist!!!")
	else:
		print("Save Deep Hedge PnL for each batch and epoch...")
		for epoch in trange(epochs, desc='Epoch loop'):
			for batch in range(num_batch_per_epoch):
				train_model_file = model_data_dir + \
					"_".join(("model", strategy_type,str(epsilon), \
					str(epoch+1), str(batch+1)))
				try:
					model_simple.load_weights(train_model_file)
				except:
					print("Weight file doesn't exist!!!")
					continue
				tmp_PnL = model_simple.predict(xtest,batch_size=test_size, verbose=0)
				loss = model_simple.evaluate(xtest,batch_size=test_size, verbose=0)
				# Numpy array format: Epoch, Batch, loss, tmp_PnL
				tmp_data = np.vstack((epoch, batch, loss, tmp_PnL))
				try:
					PnL_DH_Simple_data = np.hstack((PnL_DH_Simple_data, \
															tmp_data))
				except:
					PnL_DH_Simple_data = tmp_data

		np.save(PnL_list_file, PnL_DH_Simple_data)

	# Create the animation using FuncAnimation, convert it to HTML5 
	# video, and then display it on the Jupyter notebook.
	num_iteration_to_plot = 10**3
	def update_hist(i):
			epoch = PnL_DH_Simple_data[0,i]+ 1
			batch = PnL_DH_Simple_data[1,i]+ 1
			loss = PnL_DH_Simple_data[2,i]
			PnL = PnL_DH_Simple_data[3:,i]

			ax.cla()
			props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
			textstr = '\n'.join((
				r'$Epoch=%.1d$' % (epoch, ),
				r'$Batch=%.1d$' % (batch, ),
				r'$Loss=%.3f$' % (loss, )))
			ax.text(0.05, 0.75, textstr, transform=ax.transAxes, fontsize=11,
					verticalalignment='top', bbox=props)
			ax.hist((PnL_BS+BS_price, \
						reshape_1D(PnL+BS_price)), \
						range = (PnL_BS.min()+BS_price, PnL_BS.max()+BS_price), \
						bins=30, label=["Black-Scholes PnL", "Deep Hedging PnL"])
			ax.legend(loc="upper left")
	animation = FuncAnimation(fig_PnL, update_hist, range(num_iteration_to_plot), \
														interval = 150, cache_frame_data = False)
	display(HTML(animation.to_html5_video()))
else:
	ax.hist((PnL_BS+BS_price, reshape_1D(PnL_DH+BS_price)), \
					bins=30, label=["Black-Scholes PnL", "Deep Hedging PnL"])
	ax.legend(loc="upper left")

	fig_file = fig_dir + "fig" + "_BS_Simple_PnL" + "_" + str(epsilon) + ".png"
	fig_PnL.savefig(fig_file)

	if flag_show_fig:
		plt.show()

In [0]:
with tf.GradientTape() as tape:
  

In [0]:
#@title <font color='Blue'>**Results: Black-Scholes Delta vs Deep Hedging Delta.**</font>
for days_from_today in (1,15,29):
	if flag_plot_S_range is "percentile":
		# x-axis: [Min_S, 10th percentile, ..., 90th percentile,  Max_S]
		S_range = np.sort(S_test[0][:,days_from_today])
		idx2select = np.append(np.arange(0, len(S_range), \
						int(len(S_range)/100)), len(S_range)-1)
						
		S_range = S_range[idx2select]
	elif flag_plot_S_range is "uniform":
		min_S = S_test[0][:,days_from_today].min()
		max_S = S_test[0][:,days_from_today].max()
		S_range = np.linspace(min_S,max_S,101)
	
	# Attention: Need to transform it to be consistent with the information set.
	if information_set is "S":
		I_range =  S_range # Information set
	elif information_set is "log_S":
		I_range =  np.log(S_range)
	elif information_set is "normalized_log_S":
		I_range =  np.log(S_range/S0)
	
	# Compute NN delta (This works only for simple network!!!)
	submodel = Model(model_simple.get_layer("dense_0_" + \
							str(days_from_today)).input, model_simple.get_layer("delta_" + \
							str(days_from_today)).output)
	nn_delta = submodel.predict([I_range], batch_size=len(I_range),verbose=0)
	
	# Compute Black-Scholes delta for S_range.
	model_delta = np.zeros(S_range.shape)
	for i in range(len(S_range)):
		model_delta[i] = report.get_model_delta(instrument, s0= S_range[i], \
						calculation_date=calculation_date + days_from_today)
		
	# Create a plot of Black-Scholes delta against deep hedging delta.
	fig_delta = plt.figure(dpi= 125, facecolor='w')
	fig_delta.suptitle("Black-Scholes Delta vs Deep Hedging Delta \n", \
				fontweight="bold")
	ax_delta = fig_delta.add_subplot()
	ax_delta.set_title("Simple Network Structure with " + \
							"t=" + str(days_from_today) + ", " + \
								"epsilon=" + str(epsilon), \
								fontsize=8)
	ax_delta.set_xlabel("Price of the Underlying Asset")
	ax_delta.set_ylabel("Delta")
	ax_delta.plot(S_range, model_delta, label="Black-Scholes Delta")
	ax_delta.scatter(S_range,nn_delta, c="red", s=2, label="Deep Hedging Delta")
	ax_delta.legend()

	fig_file = fig_dir + "fig" + "_BS_Simple_Delta_" + \
							"_".join((str(epsilon),str(days_from_today))) + ".png"
	fig_delta.savefig(fig_file)

	if flag_show_fig:
		plt.show()
print("Successfully plotted Black-Scholes vs Deep Hedging Delta (Simple Network).\n")

In [0]:
#@title <font color='Blue'>**Results: Simple vs Recurrent Network**</font>

# Compute deep hedging PnL for simple network.
PnL_DH_simple = model_simple.predict(xtest,batch_size=test_size,verbose=0)

# Compute deep hedging PnL for recurrent network.
PnL_DH_recurrent = model_recurrent.predict(xtest,batch_size=test_size,verbose=0)

# Plot Simple Network PnL vs Recurrent Network PnL (with BS_price charged on both).
print("\n")
fig_nn = plt.figure(dpi= 125, facecolor='w')
fig_nn.suptitle("Simple Network PnL vs Recurrent Network PnL \n ", \
		fontweight="bold")
ax = fig_nn.add_subplot()
ax.set_title("epsilon = " + str(epsilon), fontsize=8)
ax.set_xlabel("PnL")
ax.set_ylabel("Frequency")
ax.hist((reshape_1D(PnL_DH_simple+BS_price), \
				reshape_1D(PnL_DH_recurrent+BS_price)), \
				bins=30, label=["Simple Network PnL", "Recurrent Network PnL"])
ax.legend()

fig_file = fig_dir + "fig" + "_Simple_Recurrent_PnL" + "_" + \
	str(epsilon) + "_" + str(days_from_today) + ".png"
fig_nn.savefig(fig_file)

if flag_show_fig:
	plt.show()

print("Successfully plotted Simple vs Recurrent Deep Hedging PnL.\n")

In [0]:
#@title <font color='Blue'>**Results: Black-Scholes Asymtotics**</font>
%autoreload 2

# Plot Black-Scholes model price asymptotic.
# Aim to verify our results against Figure 10 in Buehler et al (2019, QF).
strategy_type = "simple"
range_epsilon = np.power(2.0,np.arange(-5,0,1)-5)
bs_price_asym = np.zeros_like(range_epsilon)
bs_price_nonasym = np.zeros_like(range_epsilon)

# For each epsilon, compute the deep-hedging (using simple network) price.
i = 0
for epsilon_asym in range_epsilon:
	optimizer = Adam(learning_rate=lr)
 
	model_asym = Deep_Hedging(N=N, d=d+2, m=m, risk_free=risk_free, \
                      maturity=maturity).model( \
                      strategy_type=strategy_type, epsilon = epsilon_asym, \
                      loss_type=loss_type, loss_param = loss_param, \
											use_batch_norm = use_batch_norm, kernel_initializer = kernel_initializer, \
											activation_dense = activation_dense, activation_output = activation_output, \
											final_period_cost = final_period_cost)
	model_asym.compile(optimizer=optimizer)

	# Load weight from previous run (if possible) in an attempt to improve results.
	try:
		subdir_hyper_param = "/".join((str(epsilon_asym),activation_dense,activation_output,str(m),str(d),str(batch_size),""))
		model_data_dir ="./data/" + subdir_hyper_param
		
		best_model_file = model_data_dir + "_".join(("model", strategy_type,str(epsilon_asym),"{epoch}"))
		model_asym_fit = model_asym.load_weights(best_model_file)
	except:
		pass
	
	# Determine stopping criteria for this part.
	early_stopping = EarlyStopping(monitor="loss", \
						patience=5, min_delta=1e-4, restore_best_weights=True)
	reduce_lr = ReduceLROnPlateau(monitor="loss", min_lr = 1e-4, \
						factor=0.5, patience=2, min_delta=1e-3, verbose=0)
	model_checkpoint = ModelCheckpoint(best_model_file, \
						monitor="loss", save_best_only=True, \
						save_weights_only=True, verbose=0)
	callbacks = [early_stopping, reduce_lr, model_checkpoint, TqdmCallback()]
		
	if not flag_load_BS_asym:
		print("Running Simple Network with epsilon equals %2.6f.\n" % epsilon_asym)
		model_asym_fit = model_asym.fit(x=xtrain, batch_size=batch_size, epochs=epochs, \
					validation_data=(xtest, np.empty(0)), callbacks=callbacks, \
					verbose=0)
	else:
		try:
			model_asym.load_weights(best_model_file)
		except:
			print("No saved model for asymptotic Black-Scholes.\n")
	
	# Note: Using limiting results here...
	PnL_Asym = model_asym.predict(xtest,batch_size=test_size, verbose=0)
	bs_price_asym[i] = -PnL_Asym.mean()
	print("The asymptotic deep-hedging price is %2.3f.\n" % bs_price_asym[i])
	
	# Price without using the asymptotic formula.
	bs_price_nonasym[i] = model_asym.evaluate(xtest,batch_size=test_size, verbose=0)
	print("The non-asymptotic deep-hedging price is %2.3f.\n" % bs_price_nonasym[i])
	
	i += 1

# Create a plot of Black-Scholes Asymptotic - Figure 10 of Buehler et al (2019).
fig_asym = plt.figure(dpi= 125, facecolor='w')
fig_asym.suptitle("Black-Scholes Asymptotic \n", \
      fontweight="bold")
ax_asym = fig_asym.add_subplot()
ax_asym.set_xlabel("log(epsilon)")
ax_asym.set_ylabel("log(DH Price - BS Price)")

x = np.log(range_epsilon)
y = np.log(bs_price_asym - BS_price)

# Create trend (OLS) line.
[slope, const] = np.polyfit(x, y, 1)
ols_func = np.poly1d([slope, const])

ax_asym.set_title("Deep-Hedging Price (Simple Network) with different epsilons.", \
                    fontsize=8)
ax_asym.plot(x,ols_func(x),"r--", c="red", linewidth=0.8, alpha=0.5)
ax_asym.scatter(x, y, c="blue", s=2, label="log(DH Price - BS Price)")
ax_asym.legend()

props = dict(boxstyle='round', facecolor='wheat', alpha=0.1)
textstr = "y = " + "%.2f" % slope + "x + " + "%.2f" % const
ax_asym.text(0.1, 0.6, textstr, transform=ax_asym.transAxes, fontsize=11,
        verticalalignment='top', bbox=props)
				
fig_file = fig_dir + "fig" + "_BS_asym" + ".png"
fig_asym.savefig(fig_file)

if flag_show_fig:
	plt.show()

print("Successfully Verifying Black-Scholes Asymptotic.\n")